In [1]:
import phate
import scprep
import seaborn as sns
import os
import json
from PIL import Image, ImageDraw
import re
import numpy as np
import cv2
import imageio
import matplotlib.pyplot as plt
import tifffile as tiff
from tqdm import tqdm
from skimage.exposure import equalize_adapthist
from scipy.stats import stats
import matplotlib.animation as animation
import pandas as pd
import csv
import shutil
from skimage.morphology import dilation, erosion
import numpy as np
import matplotlib.pyplot as plt
import imageio
from skimage import measure
from skimage.measure import regionprops, label
from scipy.spatial import distance
import time
import datetime
from mpl_toolkits.mplot3d import Axes3D  # 3D Plotting
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, r2_score



In [4]:
import os
import json
from PIL import Image, ImageDraw
import re
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tifffile as tiff

from tqdm import tqdm
from skimage.exposure import equalize_adapthist
from scipy.stats import stats
import matplotlib.animation as animation
import pandas as pd
import csv
import shutil
from skimage.morphology import dilation, erosion
from skimage import measure
from scipy.ndimage import center_of_mass
from glob import glob
import random
from skimage.measure import regionprops, label
from scipy.spatial import distance

import imageio.v2 as imageio
from tifffile import imread


In [10]:
# # === Setup directories ===
# mask_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <-- Update this if your mask directory is located elsewhere
# output_base_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
# output_frames_dir = os.path.join(output_base_dir, "frames")
# os.makedirs(output_frames_dir, exist_ok=True)



# def analyze_mask_from_path(mask_path, frame):
#     def plot_labels(original, ax, title=None,
#                     txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
#                               'fontsize': 'x-large', 'fontweight': 'bold'},
#                     cmap='gist_earth'):
#         ax.imshow(original, cmap=cmap)
#         labels = np.unique(original)
#         print("Labels in plot_labels function:", labels)
#         ax.set_title(title)
#         for label in labels:
#             if label != 0:
#                 inds = np.argwhere(original == label)
#                 loc = inds.mean(0)
#                 ax.text(loc[1], loc[0], str(label), **txt_args)

In [11]:
# === Setup directories ===
mask_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <-- Update this if your mask directory is located elsewhere
output_base_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
output_frames_dir = os.path.join(output_base_dir, "frames")
os.makedirs(output_frames_dir, exist_ok=True)

# === Data structure to collect Frame -> Cell_IDs mapping ===
frame_cell_map = {}

# === Fixed plot size parameters ===
FIGSIZE = (8, 8)
DPI = 200  # Ensures consistent pixel size: 800x800 px

# === Main mask plotting function ===
def analyze_mask_from_path(mask_path, frame):
    def plot_labels(original, ax, title=None,
                    txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
                              'fontsize': 'x-large', 'fontweight': 'bold'},
                    cmap='gist_earth'):
        ax.imshow(original, cmap=cmap)
        labels = np.unique(original)
        labels = labels[labels != 0]  # Exclude background (label 0)
        print(f"Labels in plot_labels function (Frame {frame}):", labels)

        # Save labels per frame
        frame_cell_map[frame] = list(labels)

        ax.set_title(title)
        for label in labels:
            inds = np.argwhere(original == label)
            loc = inds.mean(0)
            ax.text(loc[1], loc[0], str(label), **txt_args)

    # Load and plot mask
    mask = imread(mask_path)
    fig, ax = plt.subplots(figsize=FIGSIZE, dpi=DPI)
    plot_labels(mask, ax, title=f"Frame {frame}")
    ax.axis('off')

    # Save figure at fixed size (no bbox cropping)
    output_path = os.path.join(output_frames_dir, f"frame_{frame:03d}.png")
    fig.savefig(output_path)
    plt.close()

# === Process all mask files ===
mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith('.tif')])
for idx, filename in enumerate(mask_files):
    mask_path = os.path.join(mask_dir, filename)
    analyze_mask_from_path(mask_path, idx)

# === Create video from frames — FIXED DIMENSIONS ===


video_output_path = os.path.join(output_base_dir, "mask_video.mp4")
with imageio.get_writer(video_output_path, fps=1, format='ffmpeg') as video_writer:
    for idx in range(len(mask_files)):
        frame_path = os.path.join(output_frames_dir, f"frame_{idx:03d}.png")
        image = imageio.imread(frame_path)
        video_writer.append_data(image)

# === Save compact CSV with Frame and Cell IDs ===
frame_cell_list = []
for frame, cell_ids in frame_cell_map.items():
    cell_ids_str = ",".join(str(cid) for cid in cell_ids)
    frame_cell_list.append({"Frame": frame, "Cell_IDs": cell_ids_str})

frame_cell_df = pd.DataFrame(frame_cell_list)
frame_cell_csv_path = os.path.join(output_base_dir, "frame_cell_list.csv")
frame_cell_df.to_csv(frame_cell_csv_path, index=False)

# Output paths
video_output_path, frame_cell_csv_path


Labels in plot_labels function (Frame 0): [ 1  2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function (Frame 1): [ 2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function (Frame 2): [ 2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function (Frame 3): [ 2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function (Frame 4): [ 2  3  4  5  6  7  8  9 10 11 12 14 22]
Labels in plot_labels function (Frame 5): [ 2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function (Frame 6): [ 2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function (Frame 7): [ 2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function (Frame 8): [ 2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function (Frame 9): [ 2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function (Frame 10): [ 2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function (Frame 11): [ 2  3  4  5  6  7  8  9 10 11 12 27]
Labels in plot_labels function (Frame 12): [

[rawvideo @ 0x38f8d080] Stream #0: not enough frames to estimate rate; consider increasing probesize


('/home/MinaHossain/EmbedTrack/Cell_Frame-Video/mask_video.mp4',
 '/home/MinaHossain/EmbedTrack/Cell_Frame-Video/frame_cell_list.csv')

In [ ]:


def analyze_cells_across_frames(mask_directory, output_directory, size_threshold=1000, max_centroid_dist=20, window_size=5):
    """Analyze and track cells across frames, store shape factors, centroids, centroid distances, and velocities in a CSV file."""
    
    start_time = time.time()

    def calculate_shape_factors(region):
        """Calculate shape factors and centroid coordinates for a single region."""
        area = region.area
        perimeter = region.perimeter
        convex_area = region.convex_area
        bounding_box_area = (region.bbox[2] - region.bbox[0]) * (region.bbox[3] - region.bbox[1])

        solidity = area / convex_area if convex_area > 0 else 0
        extent = area / bounding_box_area if bounding_box_area > 0 else 0
        circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0

        convex_mask = region.convex_image.astype(np.uint8) * 255
        contours, _ = cv2.findContours(convex_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        convex_perimeter = sum(cv2.arcLength(cnt, True) for cnt in contours) if contours else 0
        convexity = convex_perimeter / perimeter if perimeter > 0 else 0

        min_diameter = region.minor_axis_length
        max_diameter = region.major_axis_length
        elongation = 1 - (min_diameter / max_diameter) if max_diameter > 0 else 0
        compactness = (np.sqrt(4 * area / np.pi)) / max_diameter if max_diameter > 0 else 0

        centroid = region.centroid  # (y, x) format
        cx, cy = centroid[1] * 0.68, centroid[0] * 0.68  # Convert to (x, y) and apply scaling

        return [area, perimeter, solidity, extent, circularity, convexity, elongation, compactness, cx, cy]

    if not os.path.exists(mask_directory):
        raise FileNotFoundError(f"Error: The directory {mask_directory} does not exist.")

    mask_files = sorted([os.path.join(mask_directory, f) for f in os.listdir(mask_directory) if f.endswith(".tif")])
    if not mask_files:
        raise FileNotFoundError(f"No TIFF files found in {mask_directory}. Check your file paths.")

    print(f"Found {len(mask_files)} mask files.")

    all_results = []
    cell_tracks = {}  # {Cell_ID: (x, y)}
    previous_centroids = {}  # {Cell_ID: (prev_x, prev_y)}
    previous_frames = {}  # {Cell_ID: prev_frame}

    current_cell_id = 1

    for mask_path in mask_files:
        frame_str = os.path.splitext(os.path.basename(mask_path))[0]
        frame_match = re.search(r'\d+', frame_str)
        frame = int(frame_match.group()) if frame_match else None
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

        if mask is None:
            print(f"Skipping {frame}: Could not read {mask_path}.")
            continue

        labeled_mask = label(mask)
        properties = regionprops(labeled_mask)

        filtered_regions = [prop for prop in properties if prop.label != 0 and prop.area >= size_threshold]
        if not filtered_regions:
            print(f"Skipping frame {frame}: No valid cells detected.")
            continue

        used_cell_ids = set()
        for region in filtered_regions:
            shape_factors = calculate_shape_factors(region)
            centroid = (shape_factors[-2], shape_factors[-1])  # Extract (x, y) from shape factors
            matched_cell_id = None

            for cell_id, prev_centroid in cell_tracks.items():
                if cell_id not in used_cell_ids:
                    x_dist = centroid[0] - prev_centroid[0]  # Keep sign
                    y_dist = centroid[1] - prev_centroid[1]  # Keep sign
                    if abs(x_dist) <= max_centroid_dist and abs(y_dist) <= max_centroid_dist:
                        matched_cell_id = cell_id
                        used_cell_ids.add(cell_id)
                        break

            if matched_cell_id is None:
                matched_cell_id = current_cell_id
                used_cell_ids.add(matched_cell_id)
                current_cell_id += 1

            prev_x, prev_y = previous_centroids.get(matched_cell_id, (None, None))
            prev_frame = previous_frames.get(matched_cell_id, None)
            if prev_x is not None and prev_y is not None and prev_frame is not None:
                frame_diff = frame - prev_frame
                x_centroid_distance = centroid[0] - prev_x  # Keep sign
                y_centroid_distance = centroid[1] - prev_y  # Keep sign
                x_centroid_velocity = x_centroid_distance / frame_diff if frame_diff > 0 else 0
                y_centroid_velocity = y_centroid_distance / frame_diff if frame_diff > 0 else 0
            else:
                x_centroid_distance = 0
                y_centroid_distance = 0
                x_centroid_velocity = 0
                y_centroid_velocity = 0

            previous_centroids[matched_cell_id] = centroid
            previous_frames[matched_cell_id] = frame
            cell_tracks[matched_cell_id] = centroid

            all_results.append([matched_cell_id, frame] + shape_factors + [x_centroid_distance, y_centroid_distance, x_centroid_velocity, y_centroid_velocity])

    df = pd.DataFrame(all_results, columns=["Cell Number", "Frame", "Area", "Perimeter", "Extent", "Solidity", "Compactness", "Elongation", "Circularity", "Convexity", "Centroid_X", "Centroid_Y", "X_Centroid_Distance", "Y_Centroid_Distance", "X_Centroid_Velocity", "Y_Centroid_Velocity"])
    
    # Remove rows where X_Centroid_Velocity or Y_Centroid_Velocity is zero
    df = df[(df["X_Centroid_Velocity"] != 0) & (df["Y_Centroid_Velocity"] != 0)]

    frame_counts = df.groupby("Cell Number")["Frame"].nunique()
    valid_cells = frame_counts[frame_counts >= 5].index
    df = df[df["Cell Number"].isin(valid_cells)].sort_values(by=["Cell Number", "Frame"])
    
    frame_list = df.groupby("Cell Number")["Frame"].apply(list).reset_index()
    frame_list.columns = ["Cell Number", "Frames Appeared"]
    
    os.makedirs(output_directory, exist_ok=True)
    # timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    # window_size=5


    # os.makedirs(output_directory, exist_ok=True)
    # timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    original_csv_path = os.path.join(output_directory, f"Cells_Centroid_Velocity_Original_{window_size}.csv")
    df.to_csv(original_csv_path, index=False)
    
    # df.to_csv(os.path.join(output_directory, f"Cells_Centroid_Velocity_HP4_Sign_AL5-{timestamp}.csv"), index=False)
    # calculate the frame list
    frame_list.to_csv(os.path.join(output_directory, f"Cells_Frames_List_HP4_Sign_AL_{window_size}.csv"), index=False)
    # Separate Chunk for Moving Average Calculation
    moving_average_variables = ["Area", "Perimeter", "Extent", "Solidity", "Compactness", "Elongation", "Circularity", "Convexity", "Centroid_X", "Centroid_Y", "X_Centroid_Distance", "Y_Centroid_Distance", "X_Centroid_Velocity", "Y_Centroid_Velocity"]
    # window_size = 5

    for var in moving_average_variables:
        df[f"{var}_MA"] = df.groupby("Cell Number")[var].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())

    ma_csv_path = os.path.join(output_directory, f"Cells_Centroid_Velocity_MA_{window_size}.csv")
    df.to_csv(ma_csv_path, index=False)


    # frame_counts = df.groupby("Cell Number")["Frame"].nunique()
    plt.figure(figsize=(16, 10))
    plt.bar(frame_counts.index, frame_counts.values, color='blue', edgecolor='black')
    plt.xlabel("Cell ID")
    plt.ylabel("Number of Frames Appeared")
    plt.title("Histogram of Cell IDs vs Number of Frames Appeared")
    plt.xticks(rotation=90)
    plt.savefig(os.path.join(output_directory, f"Cell_IDs_vs_Frames_Distribution_Cell_WTZV.png"))
    plt.show()


    # Additional Bar Plot from "Cells_Frames_List_HP4_Sign_AL_{window_size}.csv"
    frame_list_csv_path = os.path.join(output_directory, f"Cells_Frames_List_HP4_Sign_AL_{window_size}.csv")
    if os.path.exists(frame_list_csv_path):
        frame_list_df = pd.read_csv(frame_list_csv_path)
        frame_list_df["Frames Count"] = frame_list_df["Frames Appeared"].apply(lambda x: len(eval(x)) if isinstance(x, str) else 0)

        plt.figure(figsize=(16, 10))
        plt.bar(frame_list_df["Cell Number"], frame_list_df["Frames Count"], edgecolor='black')
        plt.xlabel("Cell ID")
        plt.ylabel("Number of Frames Appeared (From CSV)")
        plt.title(f"Histogram of Cell IDs vs Number of Frames Appeared at least {window_size} times")
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.savefig(os.path.join(output_directory, f"Cell_IDs_vs_Frames_0{window_size}.png"))
        plt.show()
    else:
        print(f"Warning: {frame_list_csv_path} not found. Skipping additional bar plot.")





    print(frame_list)
    print(f"Original results saved at: {original_csv_path}")
    print(f"Results with moving averages saved at: {ma_csv_path}")
    print(f"Execution Time: {time.time() - start_time:.2f} seconds")
    
    return df





    # df.to_csv(os.path.join(output_directory, f"Cells_Centroid_Velocity_HP4_Sign_AL5-{timestamp}.csv"), index=False)
    # frame_list.to_csv(os.path.join(output_directory, f"Cells_Frames_List_HP4_Sign_AL5-{timestamp}.csv"), index=False)
    
    # print(frame_list)
    # print(f"Execution Time: {time.time() - start_time:.2f} seconds")
    # return df


#######  Example usage
mask_directory = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"
output_directory = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video/"

results_df = analyze_cells_across_frames(mask_directory, output_directory)

# Display the first few rows of the resulting DataFrame
print(results_df.head(15))
